In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, window

spark = (
    SparkSession.builder
    .appName("Fraud-Analytics")
    .config(
        "spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3,"
        "org.postgresql:postgresql:42.7.3"
    )
    .config("spark.sql.shuffle.partitions", "6")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
spark

In [3]:
fraud_raw = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "broker2:29094")
    .option("subscribe", "orders_fraud_events")
    .option("startingOffsets", "latest")
    .load()
)

fraud = (
    fraud_raw
    .selectExpr("CAST(value AS STRING) AS json")
    .selectExpr(
        """
        from_json(
            json,
            'rule STRING, order_id STRING, amount DOUBLE,
             previous_location STRING, current_location STRING, event_time STRING'
        ) AS data
        """
    )
    .select("data.*")
)


In [5]:
fraud_metrics = (
    fraud
    .groupBy("rule", "current_location")
    .agg(count("*").alias("fraud_count"))
)

(
    fraud_metrics
    .writeStream
    .foreachBatch(
        lambda df, _: (
            df.write
            .format("jdbc")
            .option("url", "jdbc:postgresql://postgres:5432/ECOMMERCE")
            .option("dbtable", "fraud_metrics")
            .option("user", "admin")
            .option("password", "admin")
            .option("driver", "org.postgresql.Driver")
            .mode("append")
            .save()
        )
    )
    .outputMode("complete")
    .start()
    .awaitTermination()
)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 